# Hands-on 07: Autoencoders for anomaly detection

This week, we will look at autoencoders for anomaly detection. 
First we will install the jetnet library and restart the kernel.

The goal is to train an autoencoder to reconstruct QCD (background) jets, which are plentiful at the LHC.
Then we will apply it to top quark (signal) jets to see if the reconstruction error is larger.
The reconstruction error can then be used as an *anomaly score* in real data.

This autoencoder architecture used is inspired by this paper: https://arxiv.org/abs/1808.08992.
![Autoencoder](https://inspirehep.net/files/70ddba8443dc13268cf2f0521c302338)

In [ ]:
! pip install --user jetnet

## Download the dataset

We will use a validation dataset of 400k jets, which is plenty for our purposes.
The full dataset is available at https://doi.org/10.5281/zenodo.2603255.

In [ ]:
import jetnet
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm


im_size = 16
jet_r = 0.8
max_jets = 50000

# download the validation data (400k jets, which is plenty for our purposes)
# full dataset is available here: https://doi.org/10.5281/zenodo.2603255
data = jetnet.datasets.TopTagging(
    jet_type="all",
    particle_features=["E", "px", "py", "pz"],
    jet_features=["type"],
    split="valid",
    data_dir="data/",
    particle_transform=jetnet.utils.cartesian_to_relEtaPhiPt,
)

## Transform and split the data

In [ ]:
import numpy as np

indices = np.random.permutation(np.arange(len(data)))[:max_jets]

In [ ]:
# transform the data
transformed_particle_data = data.particle_transform(data.particle_data[indices])

In [ ]:
# split qcd background and top quark signal
qcd_data = transformed_particle_data[data.jet_data[indices][:, 0] == 0]
top_data = transformed_particle_data[data.jet_data[indices][:, 0] == 1]

In [ ]:
from sklearn.model_selection import train_test_split

qcd_train, qcd_test = train_test_split(qcd_data, test_size=0.2, random_state=42)
_, top_test = train_test_split(top_data, test_size=0.2, random_state=42)

## Convert data to jet images

In [ ]:
import numpy as np

#  convert full dataset
qcd_train_images = np.expand_dims(jetnet.utils.to_image(qcd_train, im_size=im_size, maxR=jet_r), axis=-1)
qcd_test_images = np.expand_dims(jetnet.utils.to_image(qcd_test, im_size=im_size, maxR=jet_r), axis=-1)
top_test_images = np.expand_dims(jetnet.utils.to_image(top_test, im_size=im_size, maxR=jet_r), axis=-1)

# rescale so sum is 1 (it should be close already)
qcd_train_images = qcd_train_images / np.sum(qcd_train_images.reshape(-1, 1, 1, 1, im_size * im_size), axis=-1)
qcd_test_images = qcd_test_images / np.sum(qcd_test_images.reshape(-1, 1, 1, 1, im_size * im_size), axis=-1)
top_test_images = top_test_images / np.sum(top_test_images.reshape(-1, 1, 1, 1, im_size * im_size), axis=-1)

## Visualize the jet images

In [ ]:
def plot_jet_images(images, titles, filename="jet_image.pdf"):

    n_images = len(images)
    plt.figure(figsize=(5 * n_images, 5))

    for i, (image, title) in enumerate(zip(images, titles)):
        plt.subplot(1, n_images, i + 1)
        plt.title(title)
        plt.imshow(image, origin="lower", norm=LogNorm(vmin=1e-3, vmax=1))
        cbar = plt.colorbar()
        plt.xlabel(r"$\Delta\eta$ cell", fontsize=15)
        plt.ylabel(r"$\Delta\phi$ cell", fontsize=15)
        cbar.set_label(r"$p_T/p_T^{jet}$", fontsize=15)

    plt.tight_layout()
    plt.savefig(filename)

In [ ]:
plot_jet_images([qcd_test_images[0], top_test_images[0]], ["QCD jet image", "Top quark jet image"])

## Define the autoencoder model

In [ ]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import (
    Dense,
    Input,
    Conv2D,
    Conv2DTranspose,
    MaxPooling2D,
    Reshape,
    Flatten,
    Activation,
    UpSampling2D,
    Softmax,
)

x_in = Input(shape=(im_size, im_size, 1))
x = Conv2D(128, kernel_size=(3, 3), strides=(2, 2), activation="relu", padding="same")(x_in)
x = Conv2D(128, kernel_size=(3, 3), strides=(2, 2), activation="relu", padding="same")(x)
x = Flatten()(x)

x_enc = Dense(2, name="bottleneck")(x)

x = Dense(int(im_size * im_size / 16) * 128, activation="relu")(x_enc)
x = Reshape((int(im_size / 4), int(im_size / 4), 128))(x)
x = Conv2DTranspose(128, kernel_size=(3, 3), strides=(2, 2), activation="relu", padding="same")(x)
x = Conv2DTranspose(1, kernel_size=(3, 3), strides=(2, 2), activation="linear", padding="same")(x)
x_out = Softmax(name="softmax", axis=[-2, -3])(x)
model = Model(inputs=x_in, outputs=x_out, name="autoencoder")

model.compile(loss="mse", optimizer="adam")
model.summary()

encoder = Model(inputs=x_in, outputs=x_enc, name="encoder")
encoder.summary()

## Train the autoencoder model

In [ ]:
history = model.fit(
    qcd_train_images,
    qcd_train_images,
    batch_size=32,
    epochs=20,
    verbose=1,
    validation_data=(qcd_test_images, qcd_test_images),
)

## Reconstruction performance

In [ ]:
qcd_reco_image = model.predict(qcd_test_images[0:1]).reshape(im_size, im_size)
plot_jet_images([qcd_test_images[0], qcd_reco_image], ["Input QCD jet image", "Reconstructed QCD jet image"])

In [ ]:
top_reco_image = model.predict(top_test_images[0:1]).reshape(im_size, im_size)
plot_jet_images([top_test_images[0], top_reco_image], ["Input top quark jet image", "Reconstructed top quark jet image"])

## Anomaly detection performance

In [ ]:
qcd_reco_images = model.predict(qcd_test_images)
top_reco_images = model.predict(top_test_images)

In [ ]:
diff_qcd = np.power((qcd_reco_images - qcd_test_images), 2)
loss_qcd = np.sum(diff_qcd.reshape(-1, im_size * im_size), axis=-1)

diff_top = np.power((top_reco_images - top_test_images), 2)
loss_top = np.sum(diff_top.reshape(-1, im_size * im_size), axis=-1)

In [ ]:
plt.figure()
bins = np.arange(0, 0.2, 0.002)
plt.hist(loss_qcd, bins=bins, alpha=0.8)
plt.hist(loss_top, bins=bins, alpha=0.8)
plt.show()

## Exercices
1. Plot the ROC curve
2. Plot the 2D latent space and compare to PCA